# Практическое задание к уроку 3.
## Парсинг данных. HTML, Beautiful Soap

### Задание
**Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы).**

Получившийся список должен содержать в себе минимум:

Наименование вакансии.

Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

Ссылку на саму вакансию.

Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [41]:
# Подключаем необходимые модули и библиотеки
from bs4 import BeautifulSoup as bs
import requests
import json
from time import sleep
import pandas as pd

In [42]:
# будем парсить вакансии Дата-сайентиста
url = f"https://hh.ru/vacancies/data-scientist"
HEADERS = {
   
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

In [43]:
vacancies_list = []
i = 0
while i < 100: #Устанавливаем заведомо большое число страниц. Лишние отсеются по условию response.status_code != 200:
    sleep(0.1) #Устанавливаем искуственную задержку для исключения блокировки сайтом 
    url = 'https://hh.ru/vacancies/data-scientist?page=' + str(i)
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
      print('Парсинг завершен')
      break
    else:
        soup = bs(response.text, 'html.parser')
        items = soup.find_all('div', {'class': 'serp-item'})
        #print(f'PAGE #{i}')   
        
        for item in vacancies:
            vacancies_list.append(
                {
 # На каждой странице определяем необходимые параметры                  
                    'title': item.find('a', {'class': 'serp-item__title'}).text, # Название вакансии
                    'link': item.find('a', {'class': 'serp-item__title'})['href'], # Ссылка на вакансию
                    'salary': item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}), # Зарплата
                    'organization': item.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text, # Название компании
                    'vacancy_source': 'hh.ru',
                    
                }
            )
    i = i + 1
vacancies_list
 

Парсинг завершен


[{'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74376536?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">200 000 – 240 000 <!-- -->руб.</span>,
  'organization': 'Астор',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74257661?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">150 000 – 300 000 <!-- -->руб.</span>,
  'organization': 'НАО Финансовые системы',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data Scientist (Middle+)',
  'link': 'https://hh.ru/vacancy/76810036?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">170 000 – 240 000 <!-- -->руб.</span>,
  'organization': 'DIGINETICA',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data Scientist',
  'link': 'https://hh.ru/vacancy/76925483?query=data+scientist',
  'salary': <span clas

In [44]:
 # Приведем данные к читаемому виду
for i in vacancies_list:
        # Link
        i['link'] =  i['link'].replace('?query=data+scientist', '') 
    
        # Salary
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            # print(i['salary'])
            salary_list = i['salary'].split(' ')
            # print(salary_list)
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1].replace('\u202f', ' ')
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1].replace('\u202f', ' ')
            else:
                i['salary_min'] = salary_list[0].replace('\u202f', ' ')
                i['salary_max'] = salary_list[2].replace('\u202f', ' ')
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
        
vacancies_list


[{'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74376536',
  'organization': 'Астор',
  'vacancy_source': 'hh.ru',
  'salary_min': '200 000',
  'salary_max': '240 000',
  'salary_currency': 'руб.'},
 {'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74257661',
  'organization': 'НАО Финансовые системы',
  'vacancy_source': 'hh.ru',
  'salary_min': '150 000',
  'salary_max': '300 000',
  'salary_currency': 'руб.'},
 {'title': 'Data Scientist (Middle+)',
  'link': 'https://hh.ru/vacancy/76810036',
  'organization': 'DIGINETICA',
  'vacancy_source': 'hh.ru',
  'salary_min': '170 000',
  'salary_max': '240 000',
  'salary_currency': 'руб.'},
 {'title': 'Data Scientist',
  'link': 'https://hh.ru/vacancy/76925483',
  'organization': '7RedLines',
  'vacancy_source': 'hh.ru',
  'salary_min': '160 000',
  'salary_max': '200 000',
  'salary_currency': 'руб.'},
 {'title': 'Эконометрист/Data scientist (Junior/Middle) в консалтинг',
  'link': 'https://hh.ru/vacancy/75546364

In [45]:
# Приводим данные к датафрейму
hh_result = pd.DataFrame(vacancies_list)
hh_result

,title,link,organization,vacancy_source,salary_min,salary_max,salary_currency
0,Data scientist,https://hh.ru/vacancy/74376536,Астор,hh.ru,200 000,240 000,руб.
1,Data scientist,https://hh.ru/vacancy/74257661,НАО Финансовые системы,hh.ru,150 000,300 000,руб.
2,Data Scientist (Middle+),https://hh.ru/vacancy/76810036,DIGINETICA,hh.ru,170 000,240 000,руб.
3,Data Scientist,https://hh.ru/vacancy/76925483,7RedLines,hh.ru,160 000,200 000,руб.
4,Эконометрист/Data scientist (Junior/Middle) в ...,https://hh.ru/vacancy/75546364,ООО БСТ Менеджмент-Консалтинг,hh.ru,40 000,None,руб.
...,...,...,...,...,...,...,...
1995,Data Scientist,https://hh.ru/vacancy/67136583,Сбер. Data Science,hh.ru,None,None,None
1996,Data Scientist (Middle+),https://hh.ru/vacancy/76916638,Сбер. Data Science,hh.ru,None,None,None
1997,Data Scientist Lead в команду online-оценка ко...,https://hh.ru/vacancy/77151542,Сбер для экспертов,hh.ru,None,None,None
1998,Data Scientist,https://hh.ru/vacancy/76165484,ПАО Детский Мир,hh.ru,None,None,None


In [39]:
# Cохраняем датафрейм в файл с учетом кодировки для кирилицы
hh_result.to_csv('hh_vacansies_DS.csv', index=False, encoding='utf-16')